In [1]:
import re, os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain_core.messages import HumanMessage

from dotenv import load_dotenv
#os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

## 1. Agent 설정
LLM, Temperature, 프롬프트, 페르소나 등

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                    temperature=0,
                    request_timeout=60,
                    api_key=os.environ["OPENAI_API_KEY"])

system_prompt = '''Y
You are a three-year-old child playing with your friends and toys.
You will be shown a short story about a pretend play you are doing as a list of sentences and then a question.  
Your task is to answer using exactly one English word naming a location (e.g., “desk”, “bookshelf”).  
Do not add any extra words, punctuation, or explanation—just the single location word.
'''

integrated_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt.strip()),
    ("human", """
[Story]
{Story}

[Question]
{Question}

""")
])

## 2. 질문 하나만 테스트 해보기

In [ ]:
story = '''
1. Logan loves the beans
2. Ethan entered the office.
3. Toy robot with painted face entered the pantry.
4. Logan entered the pantry.
5. The pineapple is in the drawer.
6. Toy robot with painted face moved the pineapple to the bathtub.
7. Logan exited the pantry.
8. Ethan dislikes the tangerine
9. Toy robot with painted face exited the pantry.
10. Logan entered the pantry.
'''
qa_list = [
    {'type': 'reality', 'question':'Where is the pineapple really?', 'answer':'bathtub'},
    {'type':'memory', 'question':'Where was the pineapple at the beginning?', 'answer':'drawer'},
    {'type':'first_order', 'question':'Where will Toy robot with painted face look for the pineapple?', 'answer':'bathtub'},
    {'type':'first_order', 'question':'Where will Logan look for the pineapple?', 'answer':'bathtub'},
    {'type':'second_order', 'question':'Where does Toy robot with painted face think that Logan searches for the pineapple?', 'answer':'bathtub'},
    {'type':'second_order', 'question':'Where does Logan think that Toy robot with painted face searches for the pineapple?', 'answer':'bathtub'},
]

In [10]:
for i, qa in enumerate(qa_list, 1):
    response = llm.invoke(integrated_prompt.format_messages(
        Story=story.strip(), Question=qa['question']
    ))
    answer = response.content.strip()
    print(f"\033[94mQuestion {i} ({qa['type']}): ){qa['question']}\033[0m")
    print(f"  Model answer: {answer}, Correct answer: {qa['answer']}")

Question 1 (reality): )Where is the broccoli really?
  Model answer: pantry, Correct answer: pantry
Question 2 (memory): )Where was the broccoli at the beginning?
  Model answer: bucket, Correct answer: bucket
Question 3 (first_order): )Where will Oliver look for the broccoli?
  Model answer: pantry, Correct answer: pantry
Question 4 (first_order): )Where will Abigail look for the broccoli?
  Model answer: pantry, Correct answer: bucket
Question 5 (second_order): )Where does Oliver think that Abigail searches for the broccoli?
  Model answer: pantry, Correct answer: pantry
Question 6 (second_order): )Where does Abigail think that Oliver searches for the broccoli?
  Model answer: pantry, Correct answer: pantry


## 3. 전체 데이터셋에 실험 

In [11]:
import pandas as pd

# GitHub raw CSV URL
url = "https://raw.githubusercontent.com/beefed-up-geek/Multi-Agent/main/Experiments/ToMi/ToMi_github/tomi_dataset.csv"

# 상위 100개 story 불러오기
df = pd.read_csv(url)
df_top100 = df.head(100)

print("Loaded dataset with", len(df_top100), "stories.")

Loaded dataset with 100 stories.


In [13]:
from tqdm import tqdm                     # ← NEW: progress-bar utility

correct = {"Reality": 0, "Memory": 0,
           "First order": 0, "Second order": 0}
total   = {"Reality": 0, "Memory": 0,
           "First order": 0, "Second order": 0}

# tqdm shows progress & ETA
for _, row in tqdm(df_top100.iterrows(), total=len(df_top100), desc="Stories"):
    story = row["Story"]

    for q_col, a_col in [("Reality Question", "Reality Answer")]:
        total["Reality"] += 1
        pred = llm.invoke(
            integrated_prompt.format_messages(Story=story, Question=row[q_col])
        ).content.strip().lower()
        if pred == row[a_col].strip().lower():
            correct["Reality"] += 1

    for q_col, a_col in [("Memory Question", "Memory Answer")]:
        total["Memory"] += 1
        pred = llm.invoke(
            integrated_prompt.format_messages(Story=story, Question=row[q_col])
        ).content.strip().lower()
        if pred == row[a_col].strip().lower():
            correct["Memory"] += 1

    for q_col, a_col in [("First-Order Belief A Question", "First-Order Belief A Answer"),
                         ("First-Order Belief B Question", "First-Order Belief B Answer")]:
        total["First order"] += 1
        pred = llm.invoke(
            integrated_prompt.format_messages(Story=story, Question=row[q_col])
        ).content.strip().lower()
        if pred == row[a_col].strip().lower():
            correct["First order"] += 1

    for q_col, a_col in [("Second-Order Belief A Question", "Second-Order Belief A Answer"),
                         ("Second-Order Belief B Question", "Second-Order Belief B Answer")]:
        total["Second order"] += 1
        pred = llm.invoke(
            integrated_prompt.format_messages(Story=story, Question=row[q_col])
        ).content.strip().lower()
        if pred == row[a_col].strip().lower():
            correct["Second order"] += 1

print("\nAccuracy")
for cat in ["Reality", "Memory", "First order", "Second order"]:
    pct = correct[cat] / total[cat] * 100 if total[cat] else 0.0
    print(f"{cat:12}: {correct[cat]}/{total[cat]} = {pct:5.2f}%")


Stories: 100%|██████████| 100/100 [06:55<00:00,  4.15s/it]


Accuracy
Reality     : 96/100 = 96.00%
Memory      : 100/100 = 100.00%
First order : 161/200 = 80.50%
Second order: 109/200 = 54.50%
